**add Gabor filters?** They could enhance edge detection. Here's what we could do:

1. **Replace W pathway with Gabor bank** - Multiple orientations
2. **Add Gabor as preprocessing** - Before the network
3. **Add Gabor branch parallel to XYW** - Extra pathway

Great questions! Let me analyze both concerns:

## Will Gabor filters give better results?

**Not guaranteed.** Here's why:

| Aspect | W Pathway (Current) | Gabor Filters |
|--------|---------------------|---------------|
| **Learnable** | ✅ Yes - weights adapt to data | ❌ Fixed kernels (or partially learnable) |
| **Orientations** | 2 (H + V only) | 4-8 typically (0°, 45°, 90°, 135°...) |
| **Parameters** | Few (depthwise) | More if learnable |
| **Edge types** | Horizontal + Vertical | Multi-orientation |

**Gabor might help IF:**
- Your dataset has many diagonal/curved edges
- W pathway is underfitting directional info

**Gabor might hurt IF:**
- Fixed Gabor kernels don't match your edge statistics
- Overfitting due to more parameters

---

## Training Cost Comparison

```
Current W Pathway:
- 1×3 depthwise conv: inchannel × 1 × 3 = 3 × inchannel params
- 3×1 depthwise conv: inchannel × 3 × 1 = 3 × inchannel params
- Two 1×1 convs: ~2 × inchannel² params
- Total: ~2×inchannel² + 6×inchannel

Gabor Bank (8 orientations):
- If FIXED: 0 learnable params (but 8× more compute per forward pass)
- If LEARNABLE: 8 × kernel_size² × channels = much more params
```

| Approach | Params Increase | Compute Increase | Training Time |
|----------|-----------------|------------------|---------------|
| Fixed Gabor (8 orientations) | None | ~4× in W | +10-20% |
| Learnable Gabor | +50-100% in W | ~4× in W | +30-50% |
| Hybrid (fixed + learnable) | +25% in W | ~2× in W | +15-25% |

---

## My Recommendation

**Don't replace W entirely.** Instead, try **Option 3: Add Gabor as parallel branch**:

````python
class GaborBank(nn.Module):
    """Fixed Gabor filter bank - no learnable params, just extra features"""
    def __init__(self, in_channels, num_orientations=4):
        super().__init__()
        self.num_orientations = num_orientations
        
        # Create fixed Gabor kernels (not learnable)
        kernels = self._create_gabor_kernels(ksize=5, sigma=1.5, lambd=4.0)
        self.register_buffer('gabor_kernels', kernels)  # (num_orientations, 1, 5, 5)
        
        # Learnable 1x1 to combine Gabor responses
        self.combine = nn.Conv2d(in_channels * num_orientations, in_channels, kernel_size=1)
    
    def _create_gabor_kernels(self, ksize=5, sigma=1.5, lambd=4.0):
        import cv2
        kernels = []
        for theta in np.linspace(0, np.pi, self.num_orientations, endpoint=False):
            kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, 0.5, 0)
            kernel = kernel / kernel.sum()  # Normalize
            kernels.append(torch.from_numpy(kernel).float())
        return torch.stack(kernels).unsqueeze(1)  # (N, 1, H, W)
    
    def forward(self, x):
        B, C, H, W = x.shape
        
        # Apply each Gabor filter to each channel
        responses = []
        for i in range(self.num_orientations):
            kernel = self.gabor_kernels[i:i+1].expand(C, 1, -1, -1)
            resp = F.conv2d(x, kernel, padding=2, groups=C)
            responses.append(resp)
        
        # Concatenate and combine
        gabor_features = torch.cat(responses, dim=1)  # (B, C*num_orientations, H, W)
        return self.combine(gabor_features)


class XYWG_E(nn.Module):
    """XYW + Gabor End block (X + Y + W + G)"""
    def __init__(self, inchannel, outchannel):
        super().__init__()
        self.x_c = Xc1x1(inchannel, outchannel)
        self.y_c = Yc1x1(inchannel, outchannel)
        self.w = W(inchannel, outchannel)
        self.g = GaborBank(inchannel, num_orientations=4)  # NEW: Gabor branch
        self.fuse = nn.Conv2d(outchannel + inchannel, outchannel, kernel_size=1)  # Fuse all

    def forward(self, xc, yc, w):
        x_out = self.x_c(xc)
        y_out = self.y_c(yc)
        w_out = self.w(w)
        g_out = self.g(w)  # Gabor on same input as W
        
        # Fuse: X + Y + W, then concatenate Gabor and reduce
        xyw = x_out + y_out + w_out
        combined = torch.cat([xyw, g_out], dim=1)
        return self.fuse(combined)
````

**Why this approach:**
- ✅ Keeps W pathway (proven to work)
- ✅ Adds Gabor multi-orientation info
- ✅ Fixed Gabor = no extra learnable params
- ✅ Only 1×1 conv to fuse = minimal overhead
- ✅ Training cost increase: ~5-10%

---

## Quick Test First

Before implementing, let's check if W pathway is actually the bottleneck:

```python
# In your ablation study, check:
# If "Remove W" shows small ODS drop → W is weak → Gabor might help
# If "Remove W" shows large ODS drop → W is strong → Gabor may not help much
```

**What do your Ablation A results show for "Remove W"?** That will tell us if Gabor is worth adding.